In [1]:
!pip install faiss-cpu pypdf2

   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.2/14.6 MB 1.5 MB/s eta 0:00:10
    --------------------------------------- 0.3/14.6 MB 1.9 MB/s eta 0:00:08
   - -------------------------------------- 0.6/14.6 MB 2.5 MB/s eta 0:00:06
   - -------------------------------------- 0.6/14.6 MB 2.3 MB/s eta 0:00:06
   --- ------------------------------------ 1.1/14.6 MB 3.6 MB/s eta 0:00:04
   --- ------------------------------------ 1.2/14.6 MB 3.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.8/14.6 MB 4.4 MB/s eta 0:00:03
   ------ --------------------------------- 2.4/14.6 MB 5.0 MB/s eta 0:00:03
   -------- ------------------------------- 3.1/14.6 MB 5.9 MB/s eta 0:00:02
   --------- ------------------------------ 3.5/14.6 MB 5.8 MB/s eta 0:00:02
   ---------- ----------------------------- 4.0/14.6 MB 5.9 MB/s eta 0:00:02
   ----------

In [3]:
import faiss
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader

In [5]:
pdf_loader = PyPDFLoader("Path/to/your/pdf/document")
documents = pdf_loader.load_and_split()

In [6]:
embedding_model = OllamaEmbeddings(model='nomic-embed-text',show_progress=True)

In [7]:
document_texts = [doc.page_content for doc in documents]
documents_embeddings = embedding_model.embed_documents(document_texts)

OllamaEmbeddings: 100%|██████████| 15/15 [02:02<00:00,  8.17s/it]


In [10]:
import numpy as np

In [11]:
#create FAISS index and add the embeddings
document_embeddings_np = np.array(documents_embeddings)
dimension = document_embeddings_np.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(document_embeddings_np)

In [13]:
from langchain.docstore import InMemoryDocstore
from langchain.docstore.document import Document

In [14]:
docstore = InMemoryDocstore(dict(enumerate(documents)))
index_to_docstore_id = {i: i for i in range(len(documents))}

In [15]:
vector_store = FAISS(embedding_model, faiss_index, docstore, index_to_docstore_id)

In [16]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [26]:
# LLM from Ollama
local_model = "llama2"
llm = ChatOllama(model=local_model)

In [27]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task to phrase the user question in the best possible way in order to get the most accurate answer from the document loaded. Your response should also separately contain the precise data which was referred to in the document in order to come up with your response to the question. Make sure to refer to the exact point in the document in the form of page numbers, etc.
    Original question: {question}""",
)

In [32]:
retriever = MultiQueryRetriever.from_llm(
    vector_store.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
Ensure that you mention the page number or position in the document from which you arrived at your response. Cite the original wordings in the document along with it. Make this a separate line in the response along with your answer.
"""

prompt = ChatPromptTemplate.from_template(template)

In [33]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [34]:
chain.invoke("What are the seven technologies that half of all climate tech companies are working on?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.01it/s]


'According to the given passage, half of all climate tech companies are working on the following seven technologies:\n\n1. Recycling and waste management (9.4%)\n2. Short-duration energy storage (7.5%)\n3. Alternative proteins (7.2%)\n4. Carbon capture, utilization, and sequestration (6.3%)\n5. Long-duration energy storage (6.2%)\n6. Electricity management in buildings (5.3%)\n7. Passenger road vehicles (6.7%)\n\n(Page 1 of the document)'

In [35]:
chain.invoke("What is GreenSpace Tech by Deloitte?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  2.06it/s]


'GreenSpace Tech by Deloitte is a geography of climate tech report that analyzes the location and distribution of companies working on climate technology solutions across different regions and industries. According to the report, Colorado, Massachusetts, New York, and Texas are among the top seven states in the US for energy management, with multiple technologies tied for the lead (Pitchbook and GreenSpace Navigator/Deloitte analysis, page 13).\n\nSome standout technologies in these states include construction technology and hydrogen production in Colorado, hydro and solar in Massachusetts, HVAC in New York, and EV charging in Texas. Colorado hosts multiple startups focused on low-emission building materials and techniques such as three-dimensional (3D) printed and modular construction.\n\nIn Colorado, the state has been supporting clean hydrogen by working towards a multi-state hydrogen hub and passing a bill to help define and subsidize clean hydrogen. In Massachusetts, the Massachus